<a href="https://colab.research.google.com/github/AlcoholWolf/py-play/blob/main/San_Francisco_Crime_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# 조건 1 : 주석이 2줄 이상 있을 때  =  해당 첫 줄은 리뷰, 설명이며,    2번~ 줄은 상세 내용 입니다.
#
# 조건 2 : 주석이 1줄 있을 때  =  해당 첫 줄은 상세 내용 입니다.    더하여 2줄 이상 있더라도 전체가 상세 내용일 경우도 있습니다.
#
# 조건 3 : 만약 주석이 2줄 이상 있지만, 앞부분이 정말 많이 띄워져 있을 경우는 바로 위 내용과 연결되는 부분입니다. 
#          if: 하고 띄우는 방식과 비슷합니다.
#
# 조건 4 : 만약 명령 값(실제 코딩줄) 뒤에 주석과 번호가 달려있다면, 
#          그리고 위에 쓰여져 있는 주석 앞에도 번호가 있다면 명령 값과 해당 주석이 연결(해당 주석을 설명) 하는 주석 이라고 봐주시면 됩니다.
#
# 조건 5 : 만약 명령 값(실제 코딩줄) 뒤에 주석이 달려있다면, 코딩중 해당 명령 값에 대한 짧은 주석을 다는 것이라고 봐주시면 됩니다.
# 
# 주석 1 : 제가 조건을 설정하고 정리를 하는 편이지만 시간이 부족하므로 2차 검토까지만 하고 제출하게 될 것 같으므로, 
#          오류 또는 탈자(주석에 대한) 는 '이곳에 오탈자가 있엇다'라고 말해주시면 추후 정리할때 편합니다.

In [14]:
# Dates - 범죄 사건 발생 날짜 및 시간 정보
# Category - 범죄 사건 카테고리(이 값이 예측할 목표변수)
# Descript - 범죄 사건에 대한 자세한 설명
# DayOfWeek - 요일
# PdDistrict - 경찰서 구역 이름
# Resolution - 범죄 사건이 해결 된 방법
# Address - 범죄 사건의 대략적인 주소
# X - 경도
# Y - 위도

In [18]:
# 기본 지정
import pandas as pd
import numpy as np

# # 파일 압축 풀기 ( 이미 압축 푼 상태로 가져왓습니다. )
# !unzip '/kaggle/input/sf-crime/test.csv'
# !unzip '/kaggle/input/sf-crime/train.csv'
# !unzip '/kaggle/input/sf-crime/sampleSubmission.csv'

# .csv 파일 형식 불러오기 ( train값만 쓰는거 같아 나머지는 주석처리 )
train = pd.read_csv('./train.csv')
# test = pd.read_csv('./test.csv')
# submission = pd.read_csv('./sampleSubmission.csv')

# 최상단 5줄(분류자 포함) 로드
# head는 맨 위(분류자 또한 1번 줄)부터 5번 줄까지 반환
cthead = train.head()

# 최소 최대 평균 등의 분포도 로드
# describe는 각 열의 측정값(최소 최대 총합 평균 등등)을 반환
ctdescribe = train.describe()

# 눌이 있는지, 카운팅, 열 기준값 등등
# 데이터에 대한 대략적인 정보를 표시합니다. 
ctinfo = train.info()

# 범죄 사건의 단일값
# unique는 해당 열(기준값)의 겹치지 않은 데이터값(단일값)을 반환
ctgory = train['Category'].unique()

# 날짜 및 시간 정보를 'datetime64' 로 변경
# astype은 특정 열의 타입을 한번에 변경
ct = train['Dates'] = train['Dates'].astype('datetime64')

# 연, 월, 일, 요일, 시, 분값을 
#     새로운 컬럼(year, month, day, dayofweek, hour, minute)을 생성해서 할당하기
train['year'] = train['Dates'].dt.year
train['month'] = train['Dates'].dt.month
train['day'] = train['Dates'].dt.day
train['dayofweek'] = train['Dates'].dt.dayofweek
train['hour'] = train['Dates'].dt.hour
train['minute'] = train['Dates'].dt.minute

# 범죄가 발생한 위치를 '\n'
# 경도, 위도 값을 나타내는 X, Y 컬럼의 값을 더한 값을 담은 'X+Y' 컬럼
# X, Y의 차이 값을 담은 'X-Y' 컬럼을 생성
train['X+Y'] = train['X'] + train['Y']
train['X-Y'] = train['X'] - train['Y']

# n_days라는 컬럼을 생성, 일별 d-day를 계산하여 입력
# 날짜 값 중 가장 작은 값 dt.date.min() 의 차이를 days 형태로 출력
train['n_days'] = (train['Dates'].dt.date - train['Dates'].dt.date.min()).apply(lambda x: x.days)
# test['n_days'] = (test['Dates'].dt.date - test['Dates'].dt.date.min()).apply(lambda x: x.days)

# 변수 지정 슬롯(순서가 다른곳이어도 이곳에 변수 모으기)
# 1 | 변수y에 학습할 목표변수인 Category필드값을 담기.
y = train['Category']

# 년/월/요일/카테고리 별 범죄 발생 횟수 구하기
# sort. 정렬은 연도의 순서, 월의 순서대로 구하기 위해 호출
# counts 는 해당하는 수를 세어서 반환함.
pd.value_counts(train['year']).sort_index()
pd.value_counts(train['month']).sort_index()
pd.value_counts(train['DayOfWeek'])
pd.value_counts(train['Category'])

# 곁측치, 빈 값을 호출, 0이라면 없음 상수면 있음.
# any()로 특정 열에 대한 null값이 아닌 모든 행과 열에서 곁측치 호출
# isnull은 비어있는 값, 즉 null인 슬롯 하나당 1 반환, sum으로 각 개체 열합
ct = train.isnull().any().sum()

# 'Dates','Category','Descript','DayOfWeek','Resolution' 열 삭제
train = train.drop(['Dates','Category','Descript','DayOfWeek','Resolution'], axis=1)

train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


,PdDistrict,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y,n_days
0,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.200490,4510
1,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.200490,4510
2,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,2,23,33,-84.623949,-160.224777,4510
3,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,2,23,30,-84.626123,-160.227868,4510
4,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,2,23,30,-84.667196,-160.210279,4510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
878044,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056,2003,1,6,0,0,15,-84.744977,-160.173089,0
878045,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948,2003,1,6,0,0,1,-84.715416,-160.179311,0
878046,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266,2003,1,6,0,0,1,-84.623125,-160.183656,0
878047,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607,2003,1,6,0,0,1,-84.609924,-160.171138,0


In [ ]:
train = pd.read_csv('./train.csv')
train